In [1]:
import numpy as np
import pandas as pd
from robust_models.IRM_model.IRMClassifier import UniversalIRM
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss, f1_score

In [2]:
def nll_loss(true_labels, predicted_probs):
    total_nll = 0.0
    for i in range(len(true_labels)):
        true_class = true_labels[i]
        # Get predicted probability for the true class
        prob_true_class = predicted_probs[i, true_class]
        
        # Avoid log(0) which would be infinity
        if prob_true_class < 1e-15:
            prob_true_class = 1e-15
            
        # Calculate negative log of the probability
        sample_nll = -np.log(prob_true_class)
        total_nll += sample_nll

    
    # Return average NLL
    return total_nll / len(true_labels)

In [3]:
data = pd.read_csv('../data/electricity_source.csv', index_col=[0])
data[list(data.columns)[-1]] = data[list(data.columns)[-1]].astype(int)
X, y = data[list(data.columns)[:-1]].values, data[list(data.columns)[-1]].values
data_target = pd.read_csv('../data/electricity_target.csv', index_col=[0])
data_target[list(data_target.columns)[-1]] = data_target[list(data_target.columns)[-1]].astype(int)
data_target.reset_index(inplace=True, drop=True)
X_ood, y_ood = data_target[list(data_target.columns)[:-1]].values, data_target[list(data_target.columns)[-1]].values

In [4]:
irm_clf = UniversalIRM(task='classification', n_environments=2, 
                          lambda_irm=0.9, epochs=100, verbose=True)
irm_clf.fit(X, y)

c:\Users\Irina\anaconda3\envs\synth_shift\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Irina\anaconda3\envs\synth_shift\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\Irina\anaconda3\envs\synth_shift\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Irina\anaconda3\envs\synth_shift\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\Irina\anaconda3\envs\synth_shift\Lib\

Epoch 1/100: Loss = 0.6577
Epoch 11/100: Loss = 0.4424
Epoch 21/100: Loss = 0.4279
Early stopping at epoch 26


In [5]:
y_pred = irm_clf.predict(X_ood)
f1_score(y_ood, y_pred)

0.6163787746986454